In [1]:
# CNT — Zenodo Bundle Maker (single cell)
# Paste into your local Jupyter/Colab (with your drives mounted). No edits required unless your paths differ.

import os, sys, json, shutil, zipfile, datetime, itertools
from pathlib import Path

# === CONFIG (edit if your paths differ) ===
TS        = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
BUNDLE    = Path(r"E:\CNT\bundles") / f"cnt_zenodo_{TS}"
CALM_DIR  = Path(r"E:\CNT\artifacts\cog_alphabet_report_hybrid_v1")
OUTPOST   = Path(r"E:\cnt_outpost\artifacts")
FSS_DIR   = Path(r"E:\CNT\artifacts\cnt_fss")
CRIT_DIR  = Path(r"E:\CNT\artifacts\cnt_crit_refine")
# Optional extras (set to existing dirs if you have them)
CALM_SPEC = Path(r"E:\CNT\CALM_v1_spec")           # folder with spec + schemas + model card (if present)
CALM_KIT  = Path(r"E:\CNT\CALM_10k_KIT")           # public dashboard/evaluator kit (if present)

# === TARGET STRUCTURE ===
CALM_TARGET   = BUNDLE / "CALM_v1.1"
MPC_TARGET    = BUNDLE / "MPC1_Gridworld"
VALID_TARGET  = BUNDLE / "CNT_CriticalSuite"

# === HELPERS ===
def mkdir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def safe_copy(src: Path, dst: Path, logs: list):
    try:
        if src.is_file():
            mkdir(dst.parent)
            shutil.copy2(src, dst)
            logs.append({"status":"copied", "src": str(src), "dst": str(dst)})
        elif src.is_dir():
            if dst.exists():
                shutil.rmtree(dst)
            shutil.copytree(src, dst)
            logs.append({"status":"copied_tree", "src": str(src), "dst": str(dst)})
        else:
            logs.append({"status":"missing", "src": str(src)})
    except Exception as e:
        logs.append({"status":"error", "src": str(src), "error": repr(e)})

def zip_dir(src_dir: Path, zip_path: Path):
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
        for p in src_dir.rglob("*"):
            if p.is_file():
                zf.write(p, p.relative_to(src_dir))

def newest_dir(root: Path, pattern: str):
    cands = sorted(root.glob(pattern), key=lambda p: p.stat().st_mtime, reverse=True)
    return cands[0] if cands else None

logs = []
mkdir(BUNDLE)

# === 1) CALM v1.1 ===
mkdir(CALM_TARGET)

# Key files (if you placed them as recommended)
calm_pdf          = CALM_DIR / "CALM_Findings_v1_1.pdf"
zoom_summary_json = CALM_DIR / "analysis" / "zoom_S1_summary.json"
zoom_subjects_csv = CALM_DIR / "analysis" / "zoom_S1_subjects.csv"
calm_eval_min     = CALM_DIR / "analysis" / "calm_eval_min.py"  # sometimes saved at /analysis; adjust if elsewhere
if not calm_eval_min.exists():
    calm_eval_min = CALM_DIR / "calm_eval_min.py"

# Live demo helpers (optional)
live_decode_demo  = CALM_DIR / "live_decode_demo.py"
live_panel        = CALM_DIR / "live_panel.py"

for s, d in [
    (calm_pdf,          CALM_TARGET / "CALM_Findings_v1_1.pdf"),
    (zoom_summary_json, CALM_TARGET / "analysis" / "zoom_S1_summary.json"),
    (zoom_subjects_csv, CALM_TARGET / "analysis" / "zoom_S1_subjects.csv"),
    (calm_eval_min,     CALM_TARGET / "calm_eval_min.py"),
    (live_decode_demo,  CALM_TARGET / "live_decode_demo.py"),
    (live_panel,        CALM_TARGET / "live_panel.py"),
]:
    safe_copy(s, d, logs)

# Optional spec + kit trees
if CALM_SPEC.exists():
    safe_copy(CALM_SPEC, CALM_TARGET / "CALM_v1_spec", logs)
if CALM_KIT.exists():
    safe_copy(CALM_KIT, CALM_TARGET / "CALM_10k_KIT", logs)

# === 2) MPC-1 Gridworld ===
mkdir(MPC_TARGET)

# Find the latest ablation run that contains gridworld_ablate.png
latest_outpost = None
if OUTPOST.exists():
    # search a couple common patterns
    patterns = ["*gridworld_ablate*", "*"]
    for pat in patterns:
        cand = newest_dir(OUTPOST, f"*{pat}*")
        if cand:
            latest_outpost = cand
            break

# Known artifacts (adjust if your filenames differ)
ablate_png = None
mpc_json   = None
if latest_outpost:
    # Try to locate ablate plot by name
    candidates = list(itertools.chain(
        latest_outpost.rglob("gridworld_ablate.png"),
        OUTPOST.rglob("gridworld_ablate.png")
    ))
    ablate_png = candidates[0] if candidates else None
    # Any results JSON
    results = list(latest_outpost.rglob("*.json"))
    # Prefer one named like mpc1_results.json if present
    pref = [r for r in results if "mpc1" in r.name.lower() or "results" in r.name.lower()]
    mpc_json = pref[0] if pref else (results[0] if results else None)

if ablate_png:
    safe_copy(ablate_png, MPC_TARGET / "gridworld_ablate.png", logs)
if mpc_json:
    safe_copy(mpc_json, MPC_TARGET / "mpc1_results.json", logs)

# Example configs & seeds if present nearby
for pattern, dstname in [
    ("**/alpha_0.06*.yaml", "cfg/alpha_0.06.yaml"),
    ("**/alpha_0.08*.yaml", "cfg/alpha_0.08.yaml"),
    ("**/alpha_0.10*.yaml", "cfg/alpha_0.10.yaml"),
    ("**/seeds.txt",         "seeds.txt"),
]:
    cand = next(iter(MPC_TARGET.glob(pattern)), None)  # first, in case already copied
    if not cand and OUTPOST.exists():
        cand = next(iter(OUTPOST.rglob(pattern.replace("**/", ""))), None)
    if cand:
        safe_copy(cand, MPC_TARGET / dstname, logs)

# === 3) CNT Critical-Point Validation Suite ===
mkdir(VALID_TARGET)

# Known validation report + metrics
val_pdf = (CRIT_DIR / "cnt_validation_report.pdf") if CRIT_DIR.exists() else None
metrics = None
if CRIT_DIR.exists():
    metrics = next(iter(CRIT_DIR.rglob("metrics.json")), None)

# FSS runs folder (copy last good run if the folder exists)
fss_copy_source = None
if FSS_DIR.exists():
    # choose the newest subdir
    newest = newest_dir(FSS_DIR, "*")
    fss_copy_source = newest if newest and newest.is_dir() else FSS_DIR

if val_pdf and val_pdf.exists():
    safe_copy(val_pdf, VALID_TARGET / "cnt_validation_report.pdf", logs)
if metrics:
    safe_copy(metrics, VALID_TARGET / "metrics.json", logs)
if fss_copy_source and fss_copy_source.exists():
    safe_copy(fss_copy_source, VALID_TARGET / "fss_runs", logs)

# Also try to capture Kc crossing tables/images from cnt_crit_refine
if CRIT_DIR.exists():
    # copy small selection of crossing/summary files if they exist
    for pat in ["**/crossing*.csv", "**/Kc*.txt", "**/summary*.json", "**/*cross*.png"]:
        for fp in CRIT_DIR.rglob(pat.split("/")[-1]):
            rel = Path("refine") / fp.name
            safe_copy(fp, VALID_TARGET / rel, logs)

# === README + MANIFEST ===
readme = f"""CNT Zenodo Bundle
==================
Timestamp: {TS}

This bundle collects:
- CALM_v1.1/  (Compact Alphabet of Latent Modes)
- MPC1_Gridworld/
- CNT_CriticalSuite/  (validation & finite-size scaling)

If any files/folders are missing, see manifest.json for copy status and confirm source paths.
"""
(BUNDLE / "README.txt").write_text(readme, encoding="utf-8")
(BUNDLE / "manifest.json").write_text(json.dumps({"logs": logs}, indent=2), encoding="utf-8")

# === ZIP IT ===
ZIP_PATH = BUNDLE.with_suffix(".zip")
zip_dir(BUNDLE, ZIP_PATH)

print(f"[OK] Bundle ready:\n  Folder: {BUNDLE}\n  Zip:    {ZIP_PATH}\n")
missing = [e for e in logs if e["status"] in ("missing","error")]
if missing:
    print(f"[NOTE] {len(missing)} items were missing or errored. See manifest.json for details.")
else:
    print("[OK] All requested artifacts copied.")


[OK] Bundle ready:
  Folder: E:\CNT\bundles\cnt_zenodo_20251110-233206
  Zip:    E:\CNT\bundles\cnt_zenodo_20251110-233206.zip

[NOTE] 3 items were missing or errored. See manifest.json for details.
